# Downloading the Data based on Category:

In [46]:
import os
import time
import requests
import pandas as pd

def get_coin_data(category):
    url = f'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&category={category}&order=market_cap_desc&per_page=100&page=1&sparkline=false&locale=en'
    response = requests.get(url)
    data = response.json()
    id_list = [coin['id'] for coin in data]
    return id_list

def get_coin_chart(id):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    url = f'https://www.coingecko.com/price_charts/{id}/usd/365_days.json'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        stats_df = pd.DataFrame(data['stats'], columns=['timestamp', 'price'])
        volumes_df = pd.DataFrame(data['total_volumes'], columns=['timestamp', 'volume'])
        stats_df['timestamp'] = pd.to_datetime(stats_df['timestamp'], unit='ms')
        volumes_df['timestamp'] = pd.to_datetime(volumes_df['timestamp'], unit='ms')
        
        #remove the last row as it is the current day and the data is incomplete
        stats_df = stats_df[:-1]
        volumes_df = volumes_df[:-1]
        
        return stats_df, volumes_df
    else:
        print(f"Request for {id} failed with status code {response.status_code}")
        return None, None

def save_data(category):
    id_list = get_coin_data(category)

    for id in id_list:
        stats_df, volumes_df = get_coin_chart(id)

        if stats_df is not None and volumes_df is not None:
            os.makedirs(f'{category}/stats', exist_ok=True)
            os.makedirs(f'{category}/volume', exist_ok=True)

            stats_df.to_json(f'{category}/stats/{id}.json', orient='records', date_format='iso')
            volumes_df.to_json(f'{category}/volume/{id}.json', orient='records', date_format='iso')

        # Sleep for 2 seconds to avoid hitting rate limits
        time.sleep(2)

# Usage:
save_data('artificial-intelligence')

Request for nuco-cloud failed with status code 403


# Read the prices on to a Dataframe

In [87]:
import pandas as pd
import os

def read_price(file):
    try:
        # Read the JSON data into a DataFrame
        df = pd.read_json(file, orient='records')

        # Convert 'timestamp' to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        # Get the coin ID from the filename
        coin_id = os.path.splitext(os.path.basename(file))[0]

        # Rename the 'price' column to the coin ID
        df = df.rename(columns={'price': coin_id})

        return df

    except Exception as e:
        print(f"Error reading file {file}: {e}")
        return None

def merge_all_coins(coin_ids, base_path='artificial-intelligence/stats/'):
    # Initialize an empty DataFrame
    merged_df = pd.DataFrame()

    for coin_id in coin_ids:
        # Construct the file path
        file = os.path.join(base_path, f"{coin_id}.json")

        # Read the coin data
        coin_df = read_price(file)

        # If read_price returned None, skip this coin
        if coin_df is None:
            #print(f"Skipping {coin_id} because its data could not be read")
            continue

        # If this is the first coin, assign its data to merged_df
        if merged_df.empty:
            merged_df = coin_df
        else:
            # Otherwise, merge the coin data into merged_df
            merged_df = pd.merge(merged_df, coin_df, on='timestamp', how='outer')

    # Sort the merged dataframe by 'timestamp'
    merged_df = merged_df.sort_values('timestamp')

    return merged_df

# Usage:
coin_ids = get_coin_data('artificial-intelligence')
merged_df = merge_all_coins(coin_ids)

#filtering columns that have atleast 80% of the data
merged_df.dropna(thresh=200, axis=1, inplace=True)

df = merged_df.copy()
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

df.dropna(axis=0, inplace=True)
df

Error reading file artificial-intelligence/stats/nuco-cloud.json: File artificial-intelligence/stats/nuco-cloud.json does not exist


,render-token,bittensor,fetch-ai,akash-network,singularitynet,echelon-prime,ocean-protocol,origintrail,golem,aioz-network,...,mbd-financials,bird-money,quadency,2dai-io,smartcredit-token,epik-protocol,trace-network-labs,deepfakeai,all-in,generaitiv
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-06-10,2.291477,50.863073,0.219084,0.580056,0.246985,1.639229,0.328597,0.232233,0.188013,0.014393,...,0.000013,11.141392,0.016260,0.000885,0.694683,0.008505,0.011826,0.000655,1.849274,0.163279
2023-06-11,1.897902,44.923108,0.188870,0.504492,0.200024,1.488015,0.281082,0.206993,0.171023,0.013344,...,0.000010,10.302937,0.016260,0.000749,0.694673,0.007700,0.011333,0.000538,1.778016,0.139617
2023-06-12,1.951612,50.298651,0.182844,0.531658,0.200084,1.528897,0.292477,0.210855,0.172683,0.013529,...,0.000013,10.288757,0.015907,0.000774,0.696814,0.008161,0.016331,0.000424,1.837289,0.132541
2023-06-13,1.942500,52.815541,0.185592,0.471068,0.200338,1.487020,0.297286,0.206239,0.169169,0.013603,...,0.000013,10.368943,0.015907,0.000708,0.694622,0.008086,0.013746,0.000317,1.877049,0.119412
2023-06-14,1.926507,54.413449,0.189394,0.472823,0.206757,1.494843,0.299618,0.217890,0.169504,0.013705,...,0.000015,10.304648,0.015982,0.000704,0.693838,0.008505,0.011635,0.000462,1.802279,0.115111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-28,4.981449,315.340108,0.736102,2.484516,0.342599,10.189280,0.558799,0.639865,0.250287,0.132174,...,0.000137,14.867425,0.006333,0.001227,0.582119,0.005958,0.008605,0.001021,0.790943,0.086745
2023-12-29,4.561565,304.111760,0.686981,2.368560,0.325946,9.743751,0.525559,0.599022,0.246261,0.125577,...,0.000155,15.468657,0.006390,0.001242,0.580322,0.006802,0.008245,0.000964,0.747143,0.085382
2023-12-30,4.607544,276.881931,0.687022,2.311316,0.324139,9.445203,0.516077,0.599876,0.242054,0.138144,...,0.000173,15.247491,0.006068,0.001216,0.582703,0.006824,0.008520,0.000931,0.724286,0.083990
